<img src = "https://images2.imgbox.com/32/ac/wucGkuem_o.png" width="300">

In [ ]:
#%pip install -U -q spacy
#%pip install -q gensim

### Importando bibliotecas e criando dataframes

In [2]:
import spacy
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors
spacy.cli.download("pt_core_news_sm")

✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [3]:
artigo_treino = pd.read_csv("treino.csv")
artigo_teste = pd.read_csv("teste.csv")

In [5]:
artigo_treino.head()

,title,text,date,category,subcategory,link
0,"Após polêmica, Marine Le Pen diz que abomina n...",A candidata da direita nacionalista à Presidên...,2017-04-28,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/04/187...
1,"Macron e Le Pen vão ao 2º turno na França, em ...",O centrista independente Emmanuel Macron e a d...,2017-04-23,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/04/187...
2,"Apesar de larga vitória nas legislativas, Macr...",As eleições legislativas deste domingo (19) na...,2017-06-19,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/06/189...
3,"Governo antecipa balanço, e Alckmin anuncia qu...",O número de ocorrências de homicídios dolosos ...,2015-07-24,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2015/07...
4,"Após queda em maio, a atividade econômica sobe...","A economia cresceu 0,25% no segundo trimestre,...",2017-08-17,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/08/1...


In [6]:
artigo_teste.head()

,title,text,date,category,subcategory,link
0,Grandes irmãos,"RIO DE JANEIRO - O Brasil, cada vez menos famí...",2017-03-06,colunas,ruycastro,http://www1.folha.uol.com.br/colunas/ruycastro...
1,Haddad congela orçamento e suspende emendas de...,"O prefeito de São Paulo, Fernando Haddad (PT),...",2016-08-10,colunas,monicabergamo,http://www1.folha.uol.com.br/colunas/monicaber...
2,Proposta de reforma da Fifa tem a divulgação d...,"A Fifa divulgou, nesta quinta (10), um relatór...",2015-10-09,esporte,NaN,http://www1.folha.uol.com.br/esporte/2015/09/1...
3,"Mercado incipiente, internet das coisas conect...","Bueiros, coleiras, aparelhos hospitalares, ele...",2016-11-09,mercado,NaN,http://www1.folha.uol.com.br/mercado/2016/09/1...
4,"Mortes: Psicanalista, estudou o autismo em cri...",Toda vez que o grupo de amigos de Silvana Rabe...,2017-02-07,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2017/07...


In [7]:
nlp = spacy.load("pt_core_news_sm", disable=["paser", "ner", "tagger", "textcat"])

In [8]:
nlp

In [9]:
texto = "Rio de Janeiro é uma cidade maravilhosa"
doc = nlp(texto)

type(doc)

spacy.tokens.doc.Doc

In [10]:
textos_para_tratamento = (titulos.lower() for titulos in artigo_treino['title']) 

In [11]:
textos_para_tratamento

<generator object <genexpr> at 0x0000013971F78E40>

In [12]:
def trata_textos(doc):
    
    tokens_validos = []
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if e_valido:
            tokens_validos.append(token.text)
    if len(tokens_validos) > 2:
        return " ".join(tokens_validos)

In [13]:
texto = "Rio!!!! de Janeiro afawbqw35235 é uma cidade maravilhosa 34346v!! !!!!"
doc = nlp(texto)

trata_textos(doc)

'Rio Janeiro cidade maravilhosa'

In [14]:
type(doc[0])

spacy.tokens.token.Token

In [15]:
from time import time
t0 = time()

textos_tratados = [trata_textos(doc) for doc in nlp.pipe(textos_para_tratamento,
                                                        batch_size=1000,
                                                        n_process=3)]
tf = time() - t0
print(tf/60)

0.69798663854599


In [16]:
titulos_tratados = pd.DataFrame({"titulo": textos_tratados})

titulos_tratados.head()

,titulo
0,polêmica marine le pen abomina negacionistas h...
1,macron le pen turno frança revés siglas tradic...
2,apesar larga vitória legislativas macron terá ...
3,governo antecipa balanço alckmin anuncia queda...
4,queda maio atividade econômica sobe junho bc


In [29]:
from gensim.models import Word2Vec

w2v_modelo = Word2Vec(sg=0, window=2, vector_size=300, min_count=5, alpha=0.03, min_alpha=0.007)

In [30]:
w2v_modelo

In [32]:
print(len(titulos_tratados))

titulos_tratados = titulos_tratados.dropna().drop_duplicates()

print(len(titulos_tratados))

90000
84466


In [33]:
lista_lista_tokens = [titulo.split(" ") for titulo in titulos_tratados.titulo]

In [36]:
import logging

logging.basicConfig(format="%(asctime)s : %(message)s", level=logging.INFO)

w2v_modelo = Word2Vec(sg=0, window=2, vector_size=300, min_count=5, alpha=0.03, min_alpha=0.007)

w2v_modelo.build_vocab(lista_lista_tokens, progress_per=5000)

2022-03-26 10:25:00,581 : Word2Vec lifecycle event {'params': 'Word2Vec(vocab=0, vector_size=300, alpha=0.03)', 'datetime': '2022-03-26T10:25:00.581169', 'gensim': '4.1.2', 'python': '3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19042-SP0', 'event': 'created'}
2022-03-26 10:25:00,582 : collecting all words and their counts
2022-03-26 10:25:00,582 : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-03-26 10:25:00,589 : PROGRESS: at sentence #5000, processed 31930 words, keeping 10193 word types
2022-03-26 10:25:00,595 : PROGRESS: at sentence #10000, processed 63848 words, keeping 14989 word types
2022-03-26 10:25:00,601 : PROGRESS: at sentence #15000, processed 95753 words, keeping 18279 word types
2022-03-26 10:25:00,609 : PROGRESS: at sentence #20000, processed 127689 words, keeping 21033 word types
2022-03-26 10:25:00,616 : PROGRESS: at sentence #25000, processed 159589 words, keeping 23491 word types
2022-03-2

In [37]:
w2v_modelo.corpus_count

84466

In [39]:
w2v_modelo.train(lista_lista_tokens, total_examples=w2v_modelo.corpus_count, epochs=30)

2022-03-26 10:34:14,127 : Word2Vec lifecycle event {'msg': 'training model with 3 workers on 12924 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=2 shrink_windows=True', 'datetime': '2022-03-26T10:34:14.127423', 'gensim': '4.1.2', 'python': '3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19042-SP0', 'event': 'train'}
2022-03-26 10:34:14,471 : worker thread finished; awaiting finish of 2 more threads
2022-03-26 10:34:14,478 : worker thread finished; awaiting finish of 1 more threads
2022-03-26 10:34:14,481 : worker thread finished; awaiting finish of 0 more threads
2022-03-26 10:34:14,481 : EPOCH - 1 : training on 540242 raw words (486037 effective words) took 0.3s, 1419467 effective words/s
2022-03-26 10:34:14,828 : worker thread finished; awaiting finish of 2 more threads
2022-03-26 10:34:14,832 : worker thread finished; awaiting finish of 1 more threads
2022-03-26 10:34:14,834 : worker thread finished; aw

2022-03-26 10:34:21,425 : worker thread finished; awaiting finish of 0 more threads
2022-03-26 10:34:21,426 : EPOCH - 21 : training on 540242 raw words (486063 effective words) took 0.3s, 1458653 effective words/s
2022-03-26 10:34:21,762 : worker thread finished; awaiting finish of 2 more threads
2022-03-26 10:34:21,765 : worker thread finished; awaiting finish of 1 more threads
2022-03-26 10:34:21,767 : worker thread finished; awaiting finish of 0 more threads
2022-03-26 10:34:21,767 : EPOCH - 22 : training on 540242 raw words (486203 effective words) took 0.3s, 1458498 effective words/s
2022-03-26 10:34:22,094 : worker thread finished; awaiting finish of 2 more threads
2022-03-26 10:34:22,097 : worker thread finished; awaiting finish of 1 more threads
2022-03-26 10:34:22,102 : worker thread finished; awaiting finish of 0 more threads
2022-03-26 10:34:22,103 : EPOCH - 23 : training on 540242 raw words (486095 effective words) took 0.3s, 1484346 effective words/s
2022-03-26 10:34:22,43

(14584097, 16207260)

In [40]:
w2v_modelo.wv.most_similar("google")

[('apple', 0.5758009552955627),
 ('facebook', 0.49941691756248474),
 ('uber', 0.478995680809021),
 ('airbnb', 0.4778902530670166),
 ('amazon', 0.4686358869075775),
 ('waze', 0.4567137360572815),
 ('yahoo', 0.4483999013900757),
 ('software', 0.44797152280807495),
 ('tesla', 0.42881858348846436),
 ('walmart', 0.4254896640777588)]

In [41]:
w2v_modelo.wv.most_similar("microsoft")

[('amazon', 0.5588051676750183),
 ('unilever', 0.5300546288490295),
 ('tesla', 0.5299966335296631),
 ('lego', 0.5171110033988953),
 ('sky', 0.514362096786499),
 ('sony', 0.500957727432251),
 ('linkedin', 0.49670088291168213),
 ('walmart', 0.4919034242630005),
 ('spotify', 0.48614799976348877),
 ('braskem', 0.4860401749610901)]

In [42]:
w2v_modelo.wv.most_similar("barcelona")

[('bayern', 0.5852482914924622),
 ('madrid', 0.5567420125007629),
 ('barça', 0.5556893348693848),
 ('lazio', 0.547187864780426),
 ('united', 0.5313367247581482),
 ('chelsea', 0.5310776233673096),
 ('juventus', 0.53084397315979),
 ('psg', 0.5278435349464417),
 ('munique', 0.5120042562484741),
 ('monaco', 0.5062684416770935)]

In [43]:
w2v_modelo.wv.most_similar("messi")

[('suárez', 0.5980156064033508),
 ('cristiano', 0.5251968502998352),
 ('tevez', 0.5231528282165527),
 ('chuteiras', 0.5215114951133728),
 ('benzema', 0.5139065384864807),
 ('cavani', 0.5078448057174683),
 ('enrique', 0.49318912625312805),
 ('barcelona', 0.48997893929481506),
 ('ronaldo', 0.4858036935329437),
 ('psg', 0.4857817590236664)]

In [44]:
w2v_modelo.wv.most_similar("gm")

[('volks', 0.6646945476531982),
 ('embraer', 0.6124216318130493),
 ('chrysler', 0.5999782681465149),
 ('honda', 0.5883104801177979),
 ('braskem', 0.5850892066955566),
 ('volkswagen', 0.574164867401123),
 ('csn', 0.5720359683036804),
 ('fiat', 0.5579571723937988),
 ('mitsubishi', 0.5531455278396606),
 ('sony', 0.5508740544319153)]

In [49]:
# Cálculo de Loss

from gensim.models.callbacks import CallbackAny2Vec
from gensim.models import Word2Vec

# iniciando a chamada callback
class callback(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss após a época {}: {}'.format(self.epoch, loss))
        else:
            print('Loss após a época {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [48]:
w2v_modelo.train(lista_lista_tokens,
                total_examples=w2v_modelo.corpus_count,
                epochs = 30,
                compute_loss = True,
                callbacks=[callback()])

2022-03-26 10:40:01,535 : Effective 'alpha' higher than previous training cycles
2022-03-26 10:40:01,536 : Word2Vec lifecycle event {'msg': 'training model with 3 workers on 12924 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=2 shrink_windows=True', 'datetime': '2022-03-26T10:40:01.536263', 'gensim': '4.1.2', 'python': '3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19042-SP0', 'event': 'train'}
2022-03-26 10:40:01,852 : worker thread finished; awaiting finish of 2 more threads
2022-03-26 10:40:01,857 : worker thread finished; awaiting finish of 1 more threads
2022-03-26 10:40:01,860 : worker thread finished; awaiting finish of 0 more threads
2022-03-26 10:40:01,861 : EPOCH - 1 : training on 540242 raw words (486160 effective words) took 0.3s, 1541722 effective words/s


Loss após a época 0: 141112.515625


2022-03-26 10:40:02,182 : worker thread finished; awaiting finish of 2 more threads
2022-03-26 10:40:02,187 : worker thread finished; awaiting finish of 1 more threads
2022-03-26 10:40:02,189 : worker thread finished; awaiting finish of 0 more threads
2022-03-26 10:40:02,190 : EPOCH - 2 : training on 540242 raw words (486178 effective words) took 0.3s, 1515226 effective words/s


Loss após a época 1: 137801.171875


2022-03-26 10:40:02,514 : worker thread finished; awaiting finish of 2 more threads
2022-03-26 10:40:02,520 : worker thread finished; awaiting finish of 1 more threads
2022-03-26 10:40:02,523 : worker thread finished; awaiting finish of 0 more threads
2022-03-26 10:40:02,523 : EPOCH - 3 : training on 540242 raw words (486116 effective words) took 0.3s, 1490864 effective words/s


...

2022-03-26 10:40:10,854 : worker thread finished; awaiting finish of 2 more threads
2022-03-26 10:40:10,864 : worker thread finished; awaiting finish of 1 more threads
2022-03-26 10:40:10,866 : worker thread finished; awaiting finish of 0 more threads
2022-03-26 10:40:10,866 : EPOCH - 29 : training on 540242 raw words (486176 effective words) took 0.3s, 1564415 effective words/s


Loss após a época 28: 64718.0


2022-03-26 10:40:11,178 : worker thread finished; awaiting finish of 2 more threads
2022-03-26 10:40:11,180 : worker thread finished; awaiting finish of 1 more threads
2022-03-26 10:40:11,183 : worker thread finished; awaiting finish of 0 more threads
2022-03-26 10:40:11,183 : EPOCH - 30 : training on 540242 raw words (486049 effective words) took 0.3s, 1570730 effective words/s
2022-03-26 10:40:11,184 : Word2Vec lifecycle event {'msg': 'training on 16207260 raw words (14584476 effective words) took 9.6s, 1511698 effective words/s', 'datetime': '2022-03-26T10:40:11.183592', 'gensim': '4.1.2', 'python': '3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19042-SP0', 'event': 'train'}


Loss após a época 29: 63562.0


(14584476, 16207260)

In [50]:
#Treinamento do modelo Skip-Gram

w2v_modelo_sg = Word2Vec(sg=1, window=5, vector_size=300, min_count=5, alpha=0.03, min_alpha=0.007)

w2v_modelo_sg.build_vocab(lista_lista_tokens, progress_per=5000)

w2v_modelo_sg.train(lista_lista_tokens, total_examples=w2v_modelo_sg.corpus_count, epochs=30)

2022-03-26 10:46:00,204 : Word2Vec lifecycle event {'params': 'Word2Vec(vocab=0, vector_size=300, alpha=0.03)', 'datetime': '2022-03-26T10:46:00.204335', 'gensim': '4.1.2', 'python': '3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19042-SP0', 'event': 'created'}
2022-03-26 10:46:00,205 : collecting all words and their counts
2022-03-26 10:46:00,205 : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-03-26 10:46:00,211 : PROGRESS: at sentence #5000, processed 31930 words, keeping 10193 word types
2022-03-26 10:46:00,217 : PROGRESS: at sentence #10000, processed 63848 words, keeping 14989 word types
2022-03-26 10:46:00,223 : PROGRESS: at sentence #15000, processed 95753 words, keeping 18279 word types
2022-03-26 10:46:00,230 : PROGRESS: at sentence #20000, processed 127689 words, keeping 21033 word types
2022-03-26 10:46:00,237 : PROGRESS: at sentence #25000, processed 159589 words, keeping 23491 word types
2022-03-2

2022-03-26 10:46:09,341 : worker thread finished; awaiting finish of 2 more threads
2022-03-26 10:46:09,343 : worker thread finished; awaiting finish of 1 more threads
2022-03-26 10:46:09,346 : worker thread finished; awaiting finish of 0 more threads
2022-03-26 10:46:09,346 : EPOCH - 11 : training on 540242 raw words (486245 effective words) took 0.8s, 619492 effective words/s
2022-03-26 10:46:10,138 : worker thread finished; awaiting finish of 2 more threads
2022-03-26 10:46:10,149 : worker thread finished; awaiting finish of 1 more threads
2022-03-26 10:46:10,153 : worker thread finished; awaiting finish of 0 more threads
2022-03-26 10:46:10,154 : EPOCH - 12 : training on 540242 raw words (486198 effective words) took 0.8s, 608960 effective words/s
2022-03-26 10:46:10,933 : worker thread finished; awaiting finish of 2 more threads
2022-03-26 10:46:10,935 : worker thread finished; awaiting finish of 1 more threads
2022-03-26 10:46:10,937 : worker thread finished; awaiting finish of 0

(14584608, 16207260)

In [51]:
w2v_modelo_sg.wv.most_similar("google")

[('reguladores', 0.4349759817123413),
 ('android', 0.415201336145401),
 ('apple', 0.3931206464767456),
 ('waze', 0.3836930990219116),
 ('anunciantes', 0.38210171461105347),
 ('facebook', 0.37446630001068115),
 ('anúncios', 0.367348849773407),
 ('yahoo', 0.36428165435791016),
 ('toshiba', 0.36216428875923157),
 ('buffett', 0.35644641518592834)]

In [52]:
w2v_modelo_sg.wv.most_similar("gm")

[('metalúrgicos', 0.5394555330276489),
 ('motors', 0.5371218919754028),
 ('audi', 0.4800618886947632),
 ('honda', 0.4688883423805237),
 ('autoguiados', 0.46044448018074036),
 ('cubatão', 0.4578298330307007),
 ('volkswagen', 0.454934686422348),
 ('airbag', 0.45408231019973755),
 ('coletivas', 0.4534090459346771),
 ('bmw', 0.4530266225337982)]

In [53]:
# Salvando modelos
w2v_modelo.wv.save_word2vec_format("modelo_cbow.txt", binary=False)
w2v_modelo_sg.wv.save_word2vec_format("modelo_skipgram.txt", binary=False)

2022-03-26 10:53:32,506 : storing 12924x300 projection weights into modelo_cbow.txt
2022-03-26 10:53:34,345 : storing 12924x300 projection weights into modelo_skipgram.txt


In [69]:
# Carregando modelos
w2v_modelo_cbow = KeyedVectors.load_word2vec_format("modelo_cbow.txt")
w2v_modelo_sg = KeyedVectors.load_word2vec_format("modelo_skipgram.txt")

artigo_treino = pd.read_csv("treino.csv")
artigo_teste = pd.read_csv("teste.csv")

2022-03-26 11:20:57,501 : loading projection weights from modelo_cbow.txt
2022-03-26 11:20:59,515 : KeyedVectors lifecycle event {'msg': 'loaded (12924, 300) matrix of type float32 from modelo_cbow.txt', 'binary': False, 'encoding': 'utf8', 'datetime': '2022-03-26T11:20:59.515540', 'gensim': '4.1.2', 'python': '3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19042-SP0', 'event': 'load_word2vec_format'}
2022-03-26 11:20:59,516 : loading projection weights from modelo_skipgram.txt
2022-03-26 11:21:01,547 : KeyedVectors lifecycle event {'msg': 'loaded (12924, 300) matrix of type float32 from modelo_skipgram.txt', 'binary': False, 'encoding': 'utf8', 'datetime': '2022-03-26T11:21:01.545999', 'gensim': '4.1.2', 'python': '3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19042-SP0', 'event': 'load_word2vec_format'}


In [54]:
nlp = spacy.load("pt_core_news_sm", disable=["paser", "ner", "tagger", "textcat"])

def tokenizador(texto):
    
    doc = nlp(texto)
    tokens_validos = []
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if e_valido:
            tokens_validos.append(token.text.lower())

    return tokens_validos

texto = "Rio de Janeiro 1231231 ***** @#$ é uma cidade maravilhosa!"
tokens = tokenizador(texto)

In [55]:
print(tokens)

['rio', 'janeiro', 'cidade', 'maravilhosa']


In [65]:
def combinacao_de_vetores_por_soma(palavras, modelo):

    vetor_resultante = np.zeros((1,300))
    for pn in palavras:
        try:
            vetor_resultante += modelo.get_vector(pn)

        except KeyError:
            pass
                
    return vetor_resultante

vetor_texto = combinacao_de_vetores_por_soma(tokens, w2v_modelo_cbow)

In [66]:
print(vetor_texto.shape)

(1, 300)


In [67]:
print(vetor_texto)

[[ 0.73725584  0.15089217  1.20240411 -2.18453644  1.13463244  0.8708794
  -2.41565949 -0.83435117  1.14251167 -0.64286306  2.11320369  1.46362174
  -1.02735598  0.31004089 -0.83100927 -2.64782354 -0.35581958 -1.61779478
   0.99951565  0.95883645 -0.16870557 -2.24236462 -1.4092429   0.01914283
   0.10256626 -2.61562097  1.38824315  0.25425192  1.43732762  1.27734546
  -0.54301734 -2.74001928  0.31866984  3.04808322  2.48002577  0.09407444
  -3.21189892 -1.77520654  0.51419392 -1.17501896  0.09438811  0.72211311
   1.19624677 -1.38036093 -3.07582545  0.03012702  2.59406152 -2.47521687
   1.67371193 -1.68280038 -2.69250438 -1.67408818  1.21550885 -1.51341188
  -0.88308096  1.26416854  0.63714375 -1.94227734  1.30572701 -0.46578476
  -1.1800946  -3.59368235  1.38035727 -1.49303861  2.47762886 -0.14134818
  -0.15565002  1.01091149  0.96781203 -1.39987681  0.43151861  1.83797364
  -0.30938169  0.84137964  1.61296211  1.65670627  1.47859338  0.14305029
  -2.0172189   0.4674986  -0.84957802  

In [71]:
def matriz_vetores(textos, modelo):
    x = len(textos)
    y = 300
    matriz = np.zeros((x,y))

    for i in range(x):
        palavras = tokenizador(textos.iloc[i])
        matriz[i] = combinacao_de_vetores_por_soma(palavras, modelo)

    return matriz

matriz_vetores_treino_cbow = matriz_vetores(artigo_treino.title, w2v_modelo_cbow)
matriz_vetores_teste_cbow = matriz_vetores(artigo_teste.title, w2v_modelo_cbow)

print(matriz_vetores_treino_cbow.shape)
print(matriz_vetores_teste_cbow.shape) 

(90000, 300)
(20513, 300)


In [72]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

def classificador(modelo, x_treino, y_treino, x_teste, y_teste):

    RL = LogisticRegression(max_iter = 800)
    RL.fit(x_treino, y_treino)
    categorias = RL.predict(x_teste)
    resultados = classification_report(y_teste, categorias)
    print(resultados)
    
    return RL

RL_cbow = classificador(w2v_modelo_cbow,
                        matriz_vetores_treino_cbow,
                        artigo_treino.category,
                        matriz_vetores_teste_cbow,
                        artigo_teste.category)

              precision    recall  f1-score   support

     colunas       0.80      0.71      0.75      6103
   cotidiano       0.63      0.80      0.70      1698
     esporte       0.93      0.87      0.89      4663
   ilustrada       0.13      0.86      0.23       131
     mercado       0.84      0.78      0.81      5867
       mundo       0.74      0.83      0.78      2051

    accuracy                           0.79     20513
   macro avg       0.68      0.81      0.69     20513
weighted avg       0.82      0.79      0.80     20513



In [73]:
matriz_vetores_treino_sg = matriz_vetores(artigo_treino.title, w2v_modelo_sg)
matriz_vetores_teste_sg = matriz_vetores(artigo_teste.title, w2v_modelo_sg)

RL_sg = classificador(w2v_modelo_sg,
                      matriz_vetores_treino_sg,
                      artigo_treino.category,
                      matriz_vetores_teste_sg,
                      artigo_teste.category)

              precision    recall  f1-score   support

     colunas       0.81      0.71      0.76      6103
   cotidiano       0.64      0.81      0.71      1698
     esporte       0.93      0.87      0.90      4663
   ilustrada       0.14      0.86      0.24       131
     mercado       0.84      0.79      0.82      5867
       mundo       0.75      0.84      0.79      2051

    accuracy                           0.79     20513
   macro avg       0.69      0.81      0.70     20513
weighted avg       0.82      0.79      0.80     20513



In [74]:
# Salvando modelo

import pickle

with open("rl_cbow.pkl", "wb") as f:
    pickle.dump(RL_cbow, f)

In [75]:
with open("rl_sg.pkl", "wb") as f:
    pickle.dump(RL_sg, f)